# Sequential Bayesian Learning (SBL)
## Authors:  Author: Robert Tjarko Lange, Miro Grundei, Sam Gjisem
## Last Update: December 2018

# 0. General Introduction to the project

In this project we investigate the theoretical modelling of mismatch-negativity in the human somatosensory cortex. More specifically, we study a novel roving paradigm which is generated from a structured graphical model. This allows us to define standard and deviant stimuli in stimulus-feature-independent manner. Furthermore, sampling (as opposed to deterministic sequence generation) enables us to control for sequence-dependent cofounding factors.

The following notebook is structured as follows:

- **1. Generation of Trial Sequences based on Graphical Model**
    * We formulate a general Markov model which generates a sequence of trials (0 - low intensity, 0.5 - catch trial, 1 - high intensity)
    * The alternation probability between different observations depends upon the previous observed states as well as the hidden state/regime which is modeled as a Markov Chain.
    * We are able to increase the order of the Markov dependency in the data-generating process. This allows to vary the complexity of the sequence generation.
    
    
- **2. Modeling of different Sequential Bayesian Learning Agents**.
    * The agents process the trial sequence as it comes in. Based on different probabilistic models they update there current posterior estimate about the hidden state that drives the sampling of the observed state. 
    * Based on her current posterior estimate, we can calculate different surprise measures (e.g. Predictive, Bayesian as well as Confidence-Corrected). These surprise measures indicate how well the agent is able to infer the data-generating mechanism and the corresponding hidden states.
    * Going forward we will combine the data-generating paradigm and surprise measures with frequency data obtained from an EEG study. Thereby, we will be able to compare different models with different degrees of complexity of somatosensory adaptation and learning.
    
    
- **3. Model Comparison across Different SBL Agents and Surprise Regressors**
    * Evaluation of theoretical models in explaining the EEG frequency variation.

In [ ]:
!pip install -r requirements.txt --quiet

In [ ]:
# Import relevant sampling modules
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [1]:
import os
import time
import numpy as np

import pymc3 as pm
import theano
import matplotlib.pyplot as plt

from utils.helpers import *
from utils.plotting import *
from utils.glm_models import *

import sampling.seq_gen as sg
import sampling.seq_analysis as sa

# Import relevant SBL modules
import sbl_agents.sbl_cat_dir as sbl_cd
import sbl_agents.sbl_hmm as sbl_hmm

# Set random seed for replicability and define directories
np.random.seed(seed=1234)
results_dir = os.getcwd() + "/results/"
fig_dir = os.getcwd() + "/figures/"
data_dir = os.getcwd() + "/data/"

# 1. Generation of Trial Sequence based on Graphical Model

Our sampling paradigm is motivated by the classical roving paradigm. In such a setting a deviant is defined relative to the previous trials.

Depending on whether or not we account for second order Markov dependency also $o_{t-2}$ influences the sampling probabilities. Our generating process can be described by the following graphical model:

<img src="figures/gm_seq_gen.png" alt="drawing" width="750"/>

* Catch: $p(o_t = 0.5) = 0.05$
* Regime switch: $p(s_t \neq s_{t-1}) = 0.01$

Sample a sequence from the Graphical Model and saves it to .mat file
* 1st order Markov sequence of length 800 with following probabilities:
    * Regime 0: $p(o_t = 0|o_{t-1}=0, s_t=0), p(o_t = 1|o_{t-1}=0, s_t=0)$
    * Regime 1: $p(o_t = 0|o_{t-1}=0, s_t=1), p(o_t = 1|o_{t-1}=0, s_t=1)$
```
pythonw seq_gen.py -t 1st_temp -reg_init 0.5 -reg_change 0.01 -catch 0.05 -obs_init 0.5 -obs_change 0.35 0.65 0.65 0.35 -order 1 -v -seq
800
```

* 2nd order Markov sequence of length 800 with following probabilities:
    * Regime 0: $p(o_t = 0|o_{t-1}=0, o_{t-1}=0) = 0.35, p(o_t|o_{t-1}) = 0.65$
    * Regime 1: $p(o_t = 0|o_{t-1}=0) = 0.65, p(o_t|o_{t-1}) = 0.35$
```
pythonw seq_gen.py -t 1st_temp -reg_init 0.5 -reg_change 0.01 -catch 0.05 -obs_init 0.5 -obs_change 0.35 0.65 0.65 0.35 -order 2 -v -seq
800
```

In [ ]:
# Define parameters for sequence sampling
prob_regime_init = np.array([0.5, 0.5])  # Prob. vector for inital regime (hidden state)
prob_regime_change = 0.01  # Prob. of sampling a regime change
prob_obs_init = np.array([0.5, 0.5, 0])  # Prob. vector for inital trial/stimulus (observed state)
prob_obs_change = [0.45, 0.45, 0.05, 0.05, 0.05, 0.05, 0.45, 0.45]  # Prob. matrix for different regimes
prob_catch = 0.05  # Prob. of sampling a catch trial (independent of hidden state)

order = len(prob_obs_change)/4  # Markov order/lag dependency in the sampling scheme
seq_length = 200  # Length of sampled sequence
sample_file = "S1_200"  # Title of file saved
matlab_out = True  # Boolean - store file as .mat - otherwise .pkl file
plot_seq = True  # Booelean - plot the sampled sequence
verbose = True  # Print out the transition prob. specification and empirical statistics of the sampled seq
plot_seqs = True

In [ ]:
# Create instance of sampling class
gen_temp = sg.seq_gen(order, prob_catch, prob_regime_init, prob_regime_change,
                      prob_obs_init, prob_obs_change, verbose)

In [ ]:
# sequence = gen_temp.sample(seq_length)
# Plot the sampled sequence - when exec from command line
sg.sample_and_save(gen_temp, seq_length, sample_file,
                   matlab_out, plot_seq)

In [ ]:
if plot_seqs:
    sa.main(order=1, verbose=False, plot=True, save=False)

In [ ]:
if plot_seqs:
    sa.main(order=2, verbose=False, plot=True, save=False)

# 2. Modeling of different Sequential Bayesian Learning Agents

In [ ]:
sample_files = [["sub-01/sub-01_ses-1_run-1", "sub-01/sub-01_ses-1_run-2",
                 "sub-01/sub-01_ses-1_run-3", "sub-01/sub-01_ses-1_run-4",
                 "sub-01/sub-01_ses-1_run-5"]]

In [ ]:
sample, meta = load_obj("data/" + sample_files[0][0] + ".mat")

seq = sample[:, 2]
hidden = sample[:, 1]

prob_regime_init = meta["prob_regime_init"]
prob_obs_init = meta["prob_obs_init"]
prob_obs_change = meta["prob_obs_change"]
prob_regime_change = meta["prob_regime_change"]

In [ ]:
# General Model Settings
model_types = ["SP", "AP", "TP"]
save_results = True
verbose = True

## 2.1. Conjugate Categorical-Dirichlet Model

In [ ]:
tau = 0.

In [ ]:
# Test agents by calculating posterior after 3 timesteps
# and Surprises for t=1,2,3
for model in model_types:
    sbl_cd.test_agent(seq, hidden, tau, model, verbose)

In [ ]:
for model in model_types:
    sbl_cd.main(seq, hidden, tau, model,
                prob_regime_init, prob_obs_init, prob_obs_change,
                prob_regime_change,
                save_results, title="CD_" + model + "_" + sample_file,
                verbose=True)

In [ ]:
SP_CD = load_obj(results_dir + "CD_" + "SP" + "_" + sample_file + ".pkl")
AP_CD = load_obj(results_dir + "CD_" + "AP" + "_" + sample_file + ".pkl")
TP_CD = load_obj(results_dir + "CD_" + "TP" + "_" + sample_file + ".pkl")

In [ ]:
plot_surprise(SP_CD, AP_CD, TP_CD,
              title=r"Categorical-Dirichlet ($\tau = 0$)",
              save_pic=False)

## 2.2. Hidden Markov Model

In [ ]:
n_states = 2

In [ ]:
# Test agents by calculating posterior after 3 timesteps
# and Surprises for t=1,2,3
for model in model_types:
    sbl_hmm.test_agent(seq, hidden, n_states, model, verbose)

## 3. Model Comparison across Different SBL Agents and Surprise Regressors

In [5]:
sample_files = [["sub-01/sub-01_ses-1_run-1", "sub-01/sub-01_ses-1_run-2",
                 "sub-01/sub-01_ses-1_run-3", "sub-01/sub-01_ses-1_run-4",
                 "sub-01/sub-01_ses-1_run-5"]]

# Load in EEG Dataset
eeg_files = ["sub-01/sub-01_sbl"]
eeg_data = sio.loadmat("data/" + eeg_files[0] + ".mat")

electrodes_of_interest = {"FCz": 47, "FC2": 46, "FC4": 45, 
                          "Cz": 48, "C2": 49, "C4": 50,
                          "C6": 51, "CPz": 32, "CP2": 56,
                          "CP4": 55, "CP6": 54}

trial_coding_lookup = {11: "First Regime - Low Intensity",
                       12: "First Regime - High Intensity",
                       21: "Second Regime - Low Intensity",
                       22: "Second Regime - High Intensity",
                       33: "Catch Trial"}

# Select block and electrode for analysis
block_id = 0
elec_id = electrodes_of_interest["FCz"]
sampling_rate = 0.3
inter_stim_interval = np.array([-0.05, 0.65])

In [6]:
# Get Regressors ready!
sample, meta = load_obj("data/" + sample_files[0][block_id] + ".mat")
seq = sample[:, 2]
hidden = sample[:, 1]

In [7]:
PS_SP, BS_SP, CS_SP = sbl_cd.main(seq, hidden, tau=0, model_type="SP")
PS_AP, BS_AP, CS_AP = sbl_cd.main(seq, hidden, tau=0, model_type="AP")
PS_TP, BS_TP, CS_TP = sbl_cd.main(seq, hidden, tau=0, model_type="TP")

regressors = {"PS_SP": PS_SP, "BS_SP": BS_SP, "CS_SP": CS_SP,
              "PS_AP": PS_AP, "BS_AP": BS_AP, "CS_AP": CS_AP,
              "PS_TP": PS_TP, "BS_TP": BS_TP, "CS_TP": CS_TP}

In [8]:
# Preprocessor data to get data that we want to explain by the surprise regressors
# Return an array of shape num_trials x sampling rate
y_elec = get_electrode_data(eeg_data, block_id, elec_id, inter_stim_interval, sampling_rate)

Done selecting block and electrode specific data for [-0.05, 0.65]ms interval
Downsampled original 512 Hz Sampling Rate to 165 Hz.


# Loops to Run in computation
- 10 Electrodes
    - 20 Subjects
- 5 blocks
    - 332 Sample Point per event
        - 10 Different Models

In [ ]:
import multiprocessing
from functools import partial
from contextlib import contextmanager

np.random.seed(20)
    
def parallelize_over_samples(y_elec, regressor, model_type):    
    func = partial(run_model_estimation, y_elec=y_elec,
                   surprise_reg=regressor, model_type=model_type)
    
    sample_id = np.arange(y_elec.shape[1]).tolist()
    start = time.time()

    with multiprocessing.Pool(processes=3) as pool:
        results = pool.map(func, sample_id)
    print("Done with {} samples in secs {}".format(y_elec.shape[1], time.time() - start))

    pool.close()
    pool.join()
    return results

In [ ]:
# logger.propagate = False
import logging
logger_pymc3 = logging.getLogger('pymc3')
logger_pymc3.setLevel(logging.ERROR)

In [ ]:
parallelize_over_samples(y_elec, PS_AP, "OLS")

In [ ]:
plt.plot(y_elec[:, 200])
plt.title("Raw FCz Electrode Amplitude at Trial Time")

In [ ]:
%time free_energy = run_model_estimation(y_elec, 0, PS_AP, "OLS")

In [ ]:
plt.plot(free_energy)
plt.title("Free Energy/ELBO after ADVI Optimization")

In [ ]:
template = "{} Block | {} GLM Model | Free Energy: {} | Time: {}"

log_model_evidences = {"PS_SP": [], "BS_SP": [], "CS_SP": [],
                       "PS_AP": [], "BS_AP": [], "CS_AP": [],
                       "PS_TP": [], "BS_TP": [], "CS_TP": []}

# Something wrong with events in 5th block!
blocks = np.arange(4)
for block_id in blocks:

    sample, meta = load_obj("data/" + sample_files[0][block_id] + ".mat")
    seq, hidden = sample[:, 2], sample[:, 1]
    
    PS_SP, BS_SP, CS_SP = sbl_cd.main(seq, hidden, tau=0, model_type="SP")
    PS_AP, BS_AP, CS_AP = sbl_cd.main(seq, hidden, tau=0, model_type="AP")
    PS_TP, BS_TP, CS_TP = sbl_cd.main(seq, hidden, tau=0, model_type="TP")

    regressors = {"PS_SP": PS_SP, "BS_SP": BS_SP, "CS_SP": CS_SP,
                  "PS_AP": PS_AP, "BS_AP": BS_AP, "CS_AP": CS_AP,
                  "PS_TP": PS_TP, "BS_TP": BS_TP, "CS_TP": CS_TP}
    
    y_elec = get_electrode_data(eeg_data, block_id, elec_id)
    
    for reg_type, surprise_reg in regressors.items():
        start = time.time()
        free_energy = run_model_estimation(y_elec, surprise_reg, "OLS")
        lme_temp = free_energy[-1]
        log_model_evidences[reg_type].append(lme_temp)
        t_time = time.time() - start
        print(template.format(block_id+1, reg_type, lme_temp, t_time))
    

In [ ]:
y_labels = ["PS_SP", "BS_SP", "CS_SP",
            "PS_AP", "BS_AP", "CS_AP",
            "PS_TP", "BS_TP", "CS_TP"]
x_labels = ["Block 1", "Block 2", "Block 3", "Block 4"]

heat_data = np.array(list(log_model_evidences.values()))


fig, ax = plt.subplots()
im = ax.imshow(heat_data, cmap="Reds")

# We want to show all ticks...
ax.set_xticks(np.arange(len(x_labels)))
ax.set_yticks(np.arange(len(y_labels)))
# ... and label them with the respective list entries
ax.set_xticklabels(x_labels)
ax.set_yticklabels(y_labels)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

ax.set_title("Log Model Evidences: FCz")

* Implement Null model to compare to
* Run over all electrodes
* Make nice pipeline